In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import time
from collections import defaultdict, Counter
import json
from urllib.request import Request, urlopen
import pickle

In [216]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import shape, Point
import shapefile
from geopy import distance

import plotly.figure_factory as ff
import plotly.io as pio
import plotly as plt
pio.renderers.default = "jupyterlab"
import plotly.express as px
import plotly.graph_objects as go

In [6]:
from tqdm import tqdm

In [7]:
from utils import *

In [287]:
df = load_csv_export_dataset("export_dataframe_stage2.csv")

/Users/robclewley/.pyenv/versions/3.7.5/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning:

Columns (10,16,17,19,20,27,28,29,31,32,52,59,60,61,62,68,69,70,73,74,75,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.



<br><hr><br>

# Validate site name <--> location mapping

In [ ]:
locs = df[['SiteName', 'SiteLocation']]
len(locs)

In [39]:
locs['SiteLocation'].apply(len).value_counts()

17    53358
16     8393
15     1890
14      100
13       29
12       21
11        6
Name: SiteLocation, dtype: int64

In [32]:
all_site_names = list(locs['SiteName'].unique())
len(all_site_names)

2189

In [70]:
site_loc_map = defaultdict(set)
for site_name in all_site_names:
    this_site = df[df['SiteName'] == site_name]
    site_loc_map[site_name].update(set(this_site['latlong']))

In [13]:
all_site_names[:10]

['West Fork of Little River',
 'Apalachee River',
 'Alcovy River',
 'Bear Creek',
 'Beaverbrook Stream',
 'Herb River',
 'Bluestone Creek',
 'Braxton Creek',
 'Calls Creek',
 'Caney Creek']

Number of separate (lat, long) locations identified at the same named site

In [71]:
len(site_loc_map['West Fork of Little River'])

2

In [73]:
len(site_loc_map['Calls Creek'])

94

In [74]:
len(site_loc_map['Herb River'])

4

In [124]:
site_name = 'Calls Creek'

In [125]:
site_df = df[df['SiteName'] == site_name]

In [128]:
site_df['county'].value_counts()

Oconee    175
Name: county, dtype: int64

In [126]:
loc_pairs_site, geo_locs_site, gdf_site = get_loc_objects_from_series(site_df['SiteLocation'])

<br><hr><br>

## Load US city and town locations, as reference

The cities data is currently unused, since mapbox allows place names on the background maps.

In [88]:
with open('geodata/stanford-bx729wr3020-geojson.json') as json_file:
    cities = json.load(json_file)

In [146]:
ga_cities = {'type': 'FeatureCollection', 'features': [c for c in cities['features'] if c['properties']['state'] == 'GA']}

In [147]:
site_cities = {'type': 'FeatureCollection', 'features': [c for c in ga_cities['features'] if c['properties']['county'] == 'Oconee']}

In [142]:
len(site_cities['features'])

5

In [130]:
site_cities[0]

{'type': 'Feature',
 'id': 'bx729wr3020.15648',
 'geometry': {'type': 'Point', 'coordinates': [-83.4387722, 33.8192864]},
 'geometry_name': 'geom',
 'properties': {'gnis_id': 311456,
  'ansicode': '2405269',
  'feature': 'Civil',
  'feature2': '-999',
  'name': 'Bishop',
  'pop_2010': 224,
  'county': 'Oconee',
  'countyfips': '219',
  'state': 'GA',
  'state_fips': '13',
  'latitude': 33.8192864,
  'longitude': -83.4387722,
  'poppllat': 33.8192864,
  'poppllong': -83.4387722,
  'elev_in_m': 237,
  'elev_in_ft': 778},
 'bbox': [-83.4387722, 33.8192864, -83.4387722, 33.8192864]}